In [150]:
import random
import numpy as np
from alice import *
from ot import *
import time

__Machine Learning Algorithm__

Below is the two layer neural network we are using for comparing the two secure methods.

In [151]:
class TwoLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights 
        self.synapse0 = secure(2 * np.random.random((4,1)) - 1)

        # training
        for i in range(iterations):

            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))

            # back propagation
            layer1_error = y - layer1
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)
            
            # provide error feed by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer1_error))))

            # update
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        return layer1
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)

__Linear Secret Sharing__

Below are the functions for linear secret sharing.

In [152]:
BASE = 10

#PRECISION_INTEGRAL = 1
#PRECISION_FRACTIONAL = 6
#Q = 10000019

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

In [153]:
def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

In [154]:
def share(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def reconstruct(sharing):
    return sum(sharing) % Q

def reshare(x):
    Y = [ share(x[0]), share(x[1]), share(x[2]) ]
    return [ sum(row) % Q for row in zip(*Y) ]

In [155]:
def add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]
    
def imul(x, k):
    return [ (xi * k) % Q for xi in x ]

In [156]:
INVERSE = 104491423396290281423421247963055991507 # inverse of BASE**FRACTIONAL_PRECISION
KAPPA = 6 # leave room for five digits overflow before leakage

assert((INVERSE * BASE**PRECISION_FRACTIONAL) % Q == 1)
assert(Q > BASE**(2*PRECISION + KAPPA))

def truncate(a):
    # map to the positive range
    b = add(a, [BASE**(2*PRECISION+1), 0, 0])
    # apply mask known only by P0, and reconstruct masked b to P1 or P2
    mask = random.randrange(Q) % BASE**(PRECISION + PRECISION_FRACTIONAL + KAPPA)
    mask_low = mask % BASE**PRECISION_FRACTIONAL
    b_masked = reconstruct(add(b, [mask, 0, 0]))
    # extract lower digits
    b_masked_low = b_masked % BASE**PRECISION_FRACTIONAL
    b_low = sub(share(b_masked_low), share(mask_low))
    # remove lower digits
    c = sub(a, b_low)
    # remove extra scaling factor
    d = imul(c, INVERSE)
    return d

In [157]:
def mul(x, y):
    # local computation
    z0 = (x[0]*y[0] + x[0]*y[1] + x[1]*y[0]) % Q
    z1 = (x[1]*y[1] + x[1]*y[2] + x[2]*y[1]) % Q
    z2 = (x[2]*y[2] + x[2]*y[0] + x[0]*y[2]) % Q
    # reshare and distribute
    Z = [ share(z0), share(z1), share(z2) ]
    w = [ sum(row) % Q for row in zip(*Z) ]
    # bring precision back down from double to single
    v = truncate(w)
    return v

In [158]:
class SecureRational(object):
    
    def secure(secret):
        z = SecureRational()
        z.shares = share(encode(secret))
        return z
    
    def reveal(self):
        return decode(reconstruct(reshare(self.shares)))
    
    def __repr__(self):
        return "SecureRational(%f)" % self.reveal()
    
    def __add__(x, y):
        z = SecureRational()
        z.shares = add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = SecureRational()
        z.shares = sub(x.shares, y.shares)
        return z
    
    def __mul__(x, y):
        z = SecureRational()
        z.shares = mul(x.shares, y.shares)
        return z
    
    def __pow__(x, e):
        z = SecureRational.secure(1)
        for _ in range(e):
            z = z * x
        return z
    
    def returnSelfRational(object):
        return SecureRational()

In [159]:
class OpenRational(object):
    
    def __init__(self, secret):
        self.secret = secret
    
    def secure(secret):
        return OpenRational(secret)
    
    def reveal(self):
        return self.secret
    
    def __repr__(self):
        return "OpenRational(%f)" % self.reveal()
    
    def __add__(x, y):
        return OpenRational(x.secret + y.secret)
    
    def __sub__(x, y):
        return OpenRational(x.secret - y.secret)
    
    def __mul__(x, y):
        return OpenRational(x.secret * y.secret)
    
    def __pow__(x, e):
        z = OpenRational(1)
        for _ in range(e):
            z = z * x
        return z

In [160]:
# helper functions to map array of numbers to and from secure data type
secure = np.vectorize(lambda x: SecureRational.secure(x))
reveal = np.vectorize(lambda x: x.reveal())

In [161]:
class SigmoidMaclaurin5:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        W5  = SecureRational.secure(1/480)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [162]:
X_full = np.array([
            [0,0,0,0],
            [0,0,1,1],
            [0,1,0,1],
            [0,1,1,0],
            [1,0,0,1],
            [1,0,1,1],
            [1,1,0,0],
            [1,1,1,0]
        ])

def evaluate(network):
    for x in X_full:
        score = reveal(network.predict(secure(x)))
        prediction = 1 if score > 0.5 else 0
        print("Prediction on %s: %s (%.8f)" % (x, prediction, score))

__Garbled Circuit__

Below are the functions defining the garbled circuit

In [163]:
def execute_Garbled(data):

    for i in range(len(data)):
        temp = 0
        A = data.tolist()[i]
        #########Define the circuit###########
        on_input_gates = [[0, "AND", [0,1]], [1,"XOR", [2,3]],
                 [2, "OR", [0,3]]]
        mid_gates = [[3,"XOR", [0,1]], [4, "OR", [1,2]]]
        output_gates = [[5, "OR", [3,4]]]
        mycric = Circuit(4, on_input_gates, mid_gates, output_gates)
        ######################################
        my_input = [x[y] for x, y in zip(mycric.poss_inputs, A)]
        mycric.fire(my_input)
        j = mycric.prep_for_json()
        json_data = j
        my_keypair = list(keypair().values())
        alice = Alice(my_keypair)
        alice.setup()
        bob = Bob(1,[1])
        bob.setup()
        ##########Oblivious Transfer###########
        alice.transmit()
        if(bob.receive()):
            temp = temp + i
    
    if(temp == len(data) - 1):
        ###########All data transmitted successfully#####
        return data
    else:
        print("Please try again")

In [164]:
def dolinearshare():
    # reseed to get reproducible results
    random.seed(1)
    np.random.seed(1)

    # pick approximation
    sigmoid = SigmoidMaclaurin5()

    # train
    network = TwoLayerNetwork(sigmoid)
    network.train(secure(X_train), secure(y_train), 10000)
    network.print_weights()

    # evaluate predictions
    evaluate(network)

In [165]:
def doGarble():
# reseed to get reproducible results
    random.seed(1)
    np.random.seed(1)
    
    x_data  = execute_Garbled(X_train)
    # pick approximation
    sigmoid1 = SigmoidMaclaurin5()

    # train
    network = TwoLayerNetwork(sigmoid1)
    network.train(secure(x_data), secure(y_train), 10000)
    network.print_weights()

    # evaluate predictions
    evaluate(network)

In [168]:

start = time.time()
dolinearshare()
end = time.time()
time_for_lss = end - start
start1 = time.time()
doGarble()
end1 = time.time()
time_for_Garble = (end1 - start1)

Error: 0.005397962500000001
Error: 0.00255916
Error: 0.0016717375000000001
Error: 0.0012402175
Error: 0.0009853974999999998
Error: 0.0008172699999999999
Error: 0.0006980700000000001
Error: 0.00060921
Error: 0.000540385
Error: 0.000485495
Layer 0 weights: 
[[SecureRational(4.974140)]
 [SecureRational(-0.000830)]
 [SecureRational(-2.486043)]
 [SecureRational(-0.000831)]]
Prediction on [0 0 0 0]: 0 (0.50000000)
Prediction on [0 0 1 1]: 0 (0.00053698)
Prediction on [0 1 0 1]: 0 (0.49958473)
Prediction on [0 1 1 0]: 0 (0.00053710)
Prediction on [1 0 0 1]: 1 (5.51912503)
Prediction on [1 0 1 1]: 1 (0.99956597)
Prediction on [1 1 0 0]: 1 (5.51912717)
Prediction on [1 1 1 0]: 1 (0.99956608)
0 {0: b'O4nvQWEQ8MGr7JlGlZOmi6u61MQfd1gYYgH5anE7tBQ=', 1: b'us_D10CxjdeT14mnP1LXOtokN3w1mO4xLrphWplRsVw='} [b'gAAAAABdxVO2dflrHwiT61_v2uF0gvbtF8tN1lqyzA_UaYV7QcBRxCFJ0lGXSI6c01lT_empCmVmYav4NSRsZ4lP5htjm5ke7y0J70lqtYRhdlNm-WrZTwH6Ut1UdRM32ywSCc-_Z1PknciWDIi507i9SWHSXoWGDVQucG1sYRUQf59uLbd7NjvPVwztDh2UDvlfDb

2 {0: b'KQwVeZ7w1ca7tSTE9aqxzHKtsN4gDS3k73ZdTJyT-Uk=', 1: b'B-2tx2htm5gldifvTrL9iJxgU7tJe3oXMjTVuu5D6Xw='} [b'gAAAAABdxVO2ynI27gHwI4NmJKYtiKMdCYpxgVWVKTw_Wv792VT-3kZZ-JyuQ-xXoMlYipEXMkNX9xtNGzBD_M5a7mtVZa1iQN-XYGZszgWpZ8jJl_mmXwDKiZ0ZuTeoLRkbZhdq8_k24rNLSIGEtpHwYLWxvzt1GTITn3OzZYgGhRJTcNviqyM3vCdgM2cJC_SbgyP7SNBLyRg0wdeaKWA1UPlfQ-v_mgfuNxNPDAfKPmnT3I21eqxPvil72zTBNRsKKCnCpdRe', b'gAAAAABdxVO2lxFONEe9Z0jAoYOPbr8OQrCfCMNNkw4WgNcn1UV4c5MissfuyqOTfq9AAsv24lcZB_aOPaBMvgMU8El5R_mk6yPTSqIunCulO0eMH9CyvFkgx7OYKZHcJg6TKDHMjNekjaHwme3PTykpX3g2FLHhPlAyuw_7-eT28ut836OnwKJT1YwlEJ2PFZ2yHx-AvShTI9AgBCruZtQcqofWJOXj0bLWiBJAuSBAAwH2Ppmd_cbLFdWAMr4fd9FmOE-dRZK0', b'gAAAAABdxVO2Zp3_FGNaEzFh0Q6-KmlkMBMHGiYB9SK8ZSoSZcsKIM-GvIRFPJzwWykec5c4mwb0z9OpNRjwmiuAlUHwU1kP44SMtu6cex30De5TFvKSO_aSgHQInZs9CImm_rC396zzOFlFTUFNV6zhIw6i1u0ZF_Ef9Oj3233kov8qtigd_Mbvef69z8GNeb8t99fGUY8oHPYGpAxQgGsvt5ubemMYEtvLA0PQedVlQXZkRPdiuHis-LAri7UUkB2e2-K01Y5G', b'gAAAAABdxVO2Q6EOqQmQW-0X-q7gcqChc7CTYjDrAMABPygs8EcREOezhD1n57dmwCPHOIl

In [169]:
print(time_for_lss)
print(time_for_Garble)

25.655044078826904
25.657684564590454
